데이터 출처: 식품의약품안전처_식품영양성분데이터베이스_음식DB
https://various.foodsafetykorea.go.kr/nutrient/general/down/historyList.do


In [13]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv("20250408_음식DB.csv")

# df1에 original data copy하여 사용
df1 = df.copy()

/var/folders/9d/xx3f44kx0554j918z2q_370m0000gn/T/ipykernel_17640/243168905.py:4: DtypeWarning: Columns (45,126,144,148) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("20250408_음식DB.csv")


---------------------------------------------
식품 데이터 전처리 파이프라인
- 불필요하거나 중복되는 식품명/업체/분류 제거
- 분석에 필요한 식품만 남김
---------------------------------------------

In [16]:
# 1. 불필요한 식품명 제거
# '조리', '간편', '개입'이 포함된 식품명 제거
df1 = df1[~df1['식품명'].str.contains('조리|간편|개입', na=False)]

# 2. 중복 데이터 정리
# - 식품명이 중복된 항목 중 '식품기원명'이 '가정식(분석 함량)'이 아닌 행 제거
df1 = df1[~(df1.duplicated('식품명', keep=False) & (df1['식품기원명'] != '가정식(분석 함량)'))]

# ② 브랜드별 전처리
# - 피자: '피자_'로 시작하지만 미스터피자 아닌 항목 제거
df1 = df1[~(df1['식품명'].str.match('^피자_', na=False) & (df1['업체명'] != '미스터피자'))]

# - 피자빵: 하위 메뉴 전체 제거
df1 = df1[~df1['식품명'].str.contains('피자빵_', na=False)]

# - 버거: '버거_'로 시작하지만 맥도날드 아닌 항목 제거
df1 = df1[~(df1['식품명'].str.match('^버거_', na=False) & (df1['업체명'] != '맥도날드'))]

# ③ 특정 식품군 제거, 특정 음식으로 시작하는 음식들 지우기 
df1 = df1[~df1['식품명'].str.match('도넛', na=False)]   # 도넛 제거
df1 = df1[~df1['식품명'].str.match('마카롱', na=False)] # 마카롱 제거
df1 = df1[~df1['식품명'].str.match('롤빵', na=False)]   # 롤빵 제거
df1 = df1[~df1['식품명'].str.match('와플', na=False)]   # 와플 제거
df1 = df1[~df1['식품명'].str.match('크로플', na=False)] # 크로플 제거
df1 = df1[~df1['식품명'].str.match('빙수', na=False)] # 빙수 제거

# ④ 샌드위치 전처리
# - '샌드위치_'로 시작하면서 업체명이 '해당없음'이 아닌 경우 제거
# - '샌드위치'가 포함되지만 '샌드위치_'로 시작하지 않는 경우 제거
# - '샌드위치_'로 시작하면서 식품기원명이 '외식(분석함량)'이 아닌 경우 제거
df1 = df1[~(df1['식품명'].str.match('^샌드위치_', na=False) & (df1['업체명'] != '해당없음'))]
df1 = df1[~(df1['식품명'].str.contains('샌드위치', na=False) & ~df1['식품명'].str.match('^샌드위치_', na=False))]
df1 = df1[~(df1['식품명'].str.match('^샌드위치_', na=False) & (df1['식품기원명'] != '외식(분석함량)'))]

# ⑤ 대표식품명 기반 정제
# - '멘보샤' 대표식품명 중 실제 식품명이 '멘보샤'가 아닌 항목 제거
df1 = df1[~((df1['대표식품명'] == '멘보샤') & (df1['식품명'] != '멘보샤'))]

# - 라떼, 아이스크림 대표식품 제거
df1 = df1[~(df1['대표식품명'].str.match('라떼', na=False))]
df1 = df1[~(df1['대표식품명'].str.match('아이스크림', na=False))]

# ⑥ 식품 대분류 기반 제거
# - 음료 및 차류, 장류/양념류 제외
df1 = df1[~(df1['식품대분류명'].str.match('음료 및 차류', na=False))]
df1 = df1[~(df1['식품대분류명'].str.match('장류, 양념류', na=False))]

# ⑦ 특정 제과류 제거
# - 크로와상, 비스킷, 쿠키, 크래커, 초콜릿 관련 항목 제거
df1 = df1[~df1['식품명'].str.match('^크로와상_', na=False)]
df1 = df1[~df1['식품명'].str.match('^비스킷_', na=False)]
df1 = df1[~df1['식품명'].str.match('^쿠키_', na=False)]
df1 = df1[~df1['식품명'].str.match('^크래커_', na=False)]
df1 = df1[~df1['식품명'].str.match('^초콜릿_', na=False)]
df1 = df1[~df1['식품명'].str.contains('비스킷/쿠키/크래커', na=False)]
df1 = df1[~df1['식품명'].str.contains('기타빵', na=False)]


# 8. 식품기원명 기반 제거
# - 산업체급식 / 중고등학교급식 / 초등학교급식 제거

df1 = df1[~df1['식품기원명'].isin(['산업체급식(재료량 기반 산출 함량)', '중고등학교급식(재료량 기반 산출함량)', '초등학교급식(재료량 기반 산출 함량)'])]

# ---------------------------------------------------------
# 🎂 케이크류 데이터 정제
# - '케이크'로 시작하는 항목 중에서
#   '식품기원명'이 '가정식(분석 함량)'이 아닌 데이터 제거
#   → 가정식 기준의 영양정보만 유지하기 위함
# ---------------------------------------------------------

df1 = df1[~(
    df1['식품명'].str.match('^케이크', na=False) &
    (df1['식품기원명'] != '가정식(분석 함량)')
)]

df1 = df1[~(
    df1['식품명'].str.match('^샐러드', na=False) &
    (df1['식품기원명'] != '가정식(분석 함량)')
)]

In [17]:
df1.to_csv("20250408_음식DB_정제_테스트.csv", index=False, encoding="utf-8-sig")


